In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score


In [3]:
# Function to read dataset
def read_dataset(file_path):
    dataset = pd.read_csv(file_path, header=None)
    return dataset

In [4]:
# Function to handle missing values
def process_missing_values(train_set, test_set):
    for column in train_set.columns:
        if train_set[column].dtype == 'O':  # Categorical attribute
            sorted_vals = sorted(train_set[column].dropna().unique())
            median_val = sorted_vals[len(sorted_vals) // 2] if sorted_vals else None
        else:  # Continuous attribute
            median_val = train_set[column].median()

        if median_val is not None:
            train_set[column] = train_set[column].fillna(median_val)
            test_set[column] = test_set[column].fillna(median_val)

    return train_set, test_set

In [5]:
# Function to calculate entropy
def compute_entropy(labels):
    values, counts = np.unique(labels, return_counts=True)
    probabilities = counts / counts.sum()
    return -np.sum(probabilities * np.log2(probabilities))

In [6]:
# Function to compute Gini index
def compute_gini(labels):
    values, counts = np.unique(labels, return_counts=True)
    probabilities = counts / counts.sum()
    return 1 - np.sum(probabilities ** 2)

In [7]:
# Function to divide dataset based on an attribute
def divide_dataset(dataset, attr_index, split_value):
    left_subset = dataset[dataset.iloc[:, attr_index] <= split_value]
    right_subset = dataset[dataset.iloc[:, attr_index] > split_value]
    return left_subset, right_subset

In [8]:
# Determine best split for C4.5 algorithm
def find_best_split_c45(dataset):
    features, labels = dataset.iloc[:, :-1], dataset.iloc[:, -1]
    initial_entropy = compute_entropy(labels)
    optimal_gain_ratio, optimal_feature, optimal_split = -1, None, None

    for feature in range(features.shape[1]):
        split_values = np.unique(features.iloc[:, feature])
        for split in split_values:
            left_part, right_part = divide_dataset(dataset, feature, split)
            if len(left_part) == 0 or len(right_part) == 0:
                continue

            left_entropy = compute_entropy(left_part.iloc[:, -1])
            right_entropy = compute_entropy(right_part.iloc[:, -1])
            weighted_entropy = (len(left_part)/len(labels)) * left_entropy + (len(right_part)/len(labels)) * right_entropy
            info_gain = initial_entropy - weighted_entropy

            intrinsic_value = -((len(left_part)/len(labels)) * np.log2(len(left_part)/len(labels)) + (len(right_part)/len(labels)) * np.log2(len(right_part)/len(labels))) if len(left_part) > 0 and len(right_part) > 0 else 1
            gain_ratio = info_gain / intrinsic_value if intrinsic_value != 0 else 0

            if gain_ratio > optimal_gain_ratio:
                optimal_gain_ratio, optimal_feature, optimal_split = gain_ratio, feature, split

    return optimal_feature, optimal_split


In [9]:
# Determine best split for CART algorithm
def find_best_split_cart(dataset):
    features, labels = dataset.iloc[:, :-1], dataset.iloc[:, -1]
    optimal_gini, optimal_feature, optimal_split = float('inf'), None, None

    for feature in range(features.shape[1]):
        split_values = np.unique(features.iloc[:, feature])
        for split in split_values:
            left_part, right_part = divide_dataset(dataset, feature, split)
            if len(left_part) == 0 or len(right_part) == 0:
                continue

            weighted_gini = (len(left_part)/len(labels)) * compute_gini(left_part.iloc[:, -1]) + (len(right_part)/len(labels)) * compute_gini(right_part.iloc[:, -1])

            if weighted_gini < optimal_gini:
                optimal_gini, optimal_feature, optimal_split = weighted_gini, feature, split

    return optimal_feature, optimal_split

In [10]:
# Class for tree nodes
class TreeNode:
    def __init__(self, attribute=None, split_value=None, left_branch=None, right_branch=None, outcome=None):
        self.attribute = attribute
        self.split_value = split_value
        self.left_branch = left_branch
        self.right_branch = right_branch
        self.outcome = outcome

In [15]:
# Class to build decision trees
class DecisionTreeClassifier:
    def __init__(self, method="c45", max_depth=10):
        self.method = method
        self.max_depth = max_depth
        self.root = None

    def construct_tree(self, dataset, depth=0):
        labels = dataset.iloc[:, -1]
        if len(np.unique(labels)) == 1 or depth >= self.max_depth:
            return TreeNode(outcome=labels.iloc[0])

        feature, split = find_best_split_c45(dataset) if self.method == "c45" else find_best_split_cart(dataset)
        if feature is None:
            return TreeNode(outcome=labels.mode()[0])

        left_subset, right_subset = divide_dataset(dataset, feature, split)
        left_branch = self.construct_tree(left_subset, depth+1)
        right_branch = self.construct_tree(right_subset, depth+1)

        return TreeNode(feature, split, left_branch, right_branch)

    def train(self, dataset):
        self.root = self.construct_tree(dataset)

    def classify_instance(self, node, instance):
        if node.outcome is not None:
            return node.outcome
        if instance[node.attribute] <= node.split_value:
            return self.classify_instance(node.left_branch, instance)
        return self.classify_instance(node.right_branch, instance)

    def classify(self, features):
        return np.array([self.classify_instance(self.root, row) for _, row in features.iterrows()])


In [16]:
# Load and clean datasets
training_set = read_dataset("/content/training.data")
testing_set = read_dataset("/content/test.data")
training_set, testing_set = process_missing_values(training_set, testing_set)

In [17]:
def perform_cross_validation(data, method):
    kf = KFold(n_splits=10, shuffle=False)
    f1_scores = []

    for train_idx, val_idx in kf.split(data):
        train_fold = data.iloc[train_idx]
        val_fold = data.iloc[val_idx]

        tree = DecisionTreeClassifier(method)
        tree.train(train_fold)

        predictions = tree.classify(val_fold.iloc[:, :-1])
        f1 = f1_score(val_fold.iloc[:, -1], predictions, average="binary", pos_label="+")
        f1_scores.append(f1)

    return np.mean(f1_scores)

# Cross-validation for both models
best_f1_c45 = perform_cross_validation(training_set, "c45")
best_f1_cart = perform_cross_validation(training_set, "cart")

#Print the results
print(f"F1-score of c45: {best_f1_c45:.4f}")
print(f"F1-score of cart: {best_f1_cart:.4f}")

# Select the best model
best_method = "c45" if best_f1_c45 > best_f1_cart else "cart"
print(f"Best model after cross-validation: {best_method.upper()} (F1-score: {max(best_f1_c45, best_f1_cart):.4f})")


F1-score of c45: 0.8044
F1-score of cart: 0.8014
Best model after cross-validation: C45 (F1-score: 0.8044)


In [18]:
# Train the best model on the full training set
final_tree = DecisionTreeClassifier(best_method)
final_tree.train(training_set)

# Evaluate on the test set
final_predictions = final_tree.classify(testing_set.iloc[:, :-1])
final_accuracy = accuracy_score(testing_set.iloc[:, -1], final_predictions)
final_f1 = f1_score(testing_set.iloc[:, -1], final_predictions, average="binary", pos_label="+")
print(f"Final {best_method.upper()} Model Accuracy on Test Set: {final_accuracy:.4f}")
print(f"Final {best_method.upper()} Model F1-Score on Test Set: {final_f1:.4f}")


Final C45 Model Accuracy on Test Set: 0.8429
Final C45 Model F1-Score on Test Set: 0.8254
